# Detecção de Infarto do miocárdio usando sinais ECG
O presente documento é a representação da rede neural do artigo: "Application of deep convolutional neural network for automated detection of myocardial infarction using ECG signals".
<br>
Retirado em: <a href= "https://www.sciencedirect.com/science/article/pii/S0020025517308009"> Application of deep convolutional neural network for automated detection of myocardial infarction using ECG signals </a>
## Artigo
O artigo utiliza redes neurais convolucionais para detectar automaticamente infarto do coração. Dois modelos diferentes foram utilizados, um com ruído e outro sem. Aqui reproduziremos a arquitetura com os ruídos.
## Database
Os dados foram retirados de: <a href= "https://www.physionet.org/content/ptbdb/1.0.0/"> PTB Diagnostic ECG Database </a>. Apesar de ter 9 classes, somente 2 foram utilizadas, as de pessoas saudáveis (52) e de infarto do miocárdio(148). Outro fator importante é que são utilizados 12 leads para classificação, entretanto somento o lead II é usado. Por último, a frequência de obtenção de sinais é de 2KHz.

In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is not available.  Training on CPU ...


## Carregar os dados

In [3]:
# Carregar a tabela csv
df = pd.read_csv("batimentos.csv", header= None)
df.head


<bound method NDFrame.head of             0         1         2         3         4         5         6    \
0      436.0000  426.0000  422.0000    0.4200  416.0000  418.0000    0.4175   
1      444.0000  458.0000  455.0000    0.4475    0.4485  443.0000  439.0000   
2      436.0000  441.0000  436.0000  432.0000  432.0000    0.4235    0.4200   
3      407.0000  404.0000    0.4135    0.4205    0.4185  413.0000  414.0000   
4        0.4095    0.3965  404.0000    0.4295  437.0000    0.4375  435.0000   
...         ...       ...       ...       ...       ...       ...       ...   
18179    0.1255    0.1165    0.1095  105.0000    0.1015    0.0995    0.1035   
18180   43.0000   61.0000    0.0475   47.0000   85.0000    0.0800    0.0735   
18181    0.1025    0.1155  103.0000    0.0915  106.0000    0.1105    0.0775   
18182    0.1135   71.0000    0.0555    0.0635    0.0715    0.0875   77.0000   
18183    0.1105    0.1015    0.0995   89.0000   77.0000    0.0785    0.0855   

            7        

In [4]:
total_size = df.shape[0]

train_size = int(0.63 * total_size)
validation_size = int(0.27 * total_size)
test_size = total_size - (validation_size + train_size)

print(f"Train size: {train_size}, Validation size: {validation_size},\
 Test size: {test_size}")


Train size: 11455, Validation size: 4909, Test size: 1820


In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,642,643,644,645,646,647,648,649,650,651
0,436.0000,426.0000,422.0000,0.4200,416.0000,418.0000,0.4175,413.0000,0.4105,408.0000,...,339.0000,0.3305,336.0000,0.3485,344.0000,0.3435,344.0000,0.3355,334.0000,1.0
1,444.0000,458.0000,455.0000,0.4475,0.4485,443.0000,439.0000,0.4255,0.4185,422.0000,...,365.0000,361.0000,356.0000,0.3625,352.0000,345.0000,0.3405,0.3445,339.0000,1.0
2,436.0000,441.0000,436.0000,432.0000,432.0000,0.4235,0.4200,422.0000,0.4325,0.4305,...,0.3325,327.0000,324.0000,0.3265,336.0000,339.0000,0.3405,0.3375,332.0000,1.0
3,407.0000,404.0000,0.4135,0.4205,0.4185,413.0000,414.0000,406.0000,404.0000,418.0000,...,332.0000,0.3385,327.0000,323.0000,0.3375,0.3505,0.3400,0.3285,0.3205,1.0
4,0.4095,0.3965,404.0000,0.4295,437.0000,0.4375,435.0000,0.4325,438.0000,439.0000,...,352.0000,358.0000,0.3465,341.0000,347.0000,345.0000,337.0000,344.0000,0.3515,1.0


In [6]:
header = df.columns

In [7]:
header

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            642, 643, 644, 645, 646, 647, 648, 649, 650, 651],
           dtype='int64', length=652)

In [8]:
from torch.utils.data.dataset import Dataset

class Batimentos(Dataset):
  def __init__(self):
    xy = df
    self.x = torch.tensor(df[header[:-1]].values, dtype= torch.float32)
    self.y = torch.tensor((df[header[-1]].values), dtype= int)
    self.tamanho = xy.shape[0]
  
  def __len__(self):
    return self.tamanho
  
  def __getitem__(self, index):
    return self.x[index], self.y[index]

In [9]:
dataset = Batimentos()
dataset[1]

(tensor([4.4400e+02, 4.5800e+02, 4.5500e+02, 4.4750e-01, 4.4850e-01, 4.4300e+02,
         4.3900e+02, 4.2550e-01, 4.1850e-01, 4.2200e+02, 4.1250e-01, 4.1850e-01,
         4.2150e-01, 4.1250e-01, 4.0950e-01, 4.0150e-01, 3.8950e-01, 3.8900e+02,
         3.9600e+02, 3.9500e+02, 3.8550e-01, 3.7950e-01, 3.7650e-01, 3.8000e-01,
         3.8000e-01, 3.6950e-01, 3.6350e-01, 3.6700e+02, 3.7750e-01, 3.8100e+02,
         3.6700e+02, 3.5750e-01, 3.5400e+02, 3.4800e+02, 3.4300e+02, 3.4400e+02,
         3.6200e+02, 3.7300e+02, 3.6550e-01, 3.6300e+02, 3.6300e+02, 3.6350e-01,
         3.6800e+02, 3.6450e-01, 3.5900e+02, 3.5750e-01, 3.5500e+02, 3.4950e-01,
         3.4100e+02, 3.4300e+02, 3.4750e-01, 3.5000e-01, 3.5450e-01, 3.5150e-01,
         3.4600e+02, 3.4050e-01, 3.4150e-01, 3.5100e+02, 3.5200e+02, 3.4600e+02,
         3.5550e-01, 3.5750e-01, 3.5900e+02, 3.6650e-01, 3.6100e+02, 3.5950e-01,
         3.5650e-01, 3.5050e-01, 3.4850e-01, 3.4400e+02, 3.4400e+02, 3.3950e-01,
         3.3550e-01, 3.3900e

In [15]:
from torch.utils.data import DataLoader


batch_size = 10
train_data, validation, test_data = torch.utils.data.random_split(dataset, [train_size, validation_size, test_size])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle= True)
valid_loader = torch.utils.data.DataLoader(validation, batch_size=batch_size, shuffle= True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle= False)

In [18]:
data, label = next(iter(train_loader))
data.size(0)

10

In [20]:
len(train_loader.dataset)

11455

## Arquitetura

In [11]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Entrada de um vetor de 651 dados 
        self.conv1 = nn.Conv1d(1, 3, 102, padding=1)
        # Saida 550x3
        # Entrada 275x3
        self.conv2 = nn.Conv1d(3, 10, 24, padding=1)
        # Saida 252x10
        # Entrada 126x10
        self.conv3 = nn.Conv1d(10, 10, 11, padding=1)
        # Saida 116x10
        # Entrada 58x10
        self.conv4 = nn.Conv1d(10, 10, 9, padding=1)
        # Saida 50x10

        self.pool = nn.MaxPool1d(2, stride= 2)

        # linear layer (25*10 -> 500)
        self.fc1 = nn.Linear(25*10, 30)
        # linear layer (30 -> 10)
        self.fc2 = nn.Linear(30, 10)
        # linear layer (500 -> 10)
        self.fc3 = nn.Linear(10, 2)


    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = self.pool(F.leaky_relu(self.conv3(x)))
        x = self.pool(F.leaky_relu(self.conv4(x)))
        
        # flatten image input
        x = x.view(-1, 25*10)

        # FC com leaky_relu
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.softmax(self.fc3(x))

        return x

# create a complete CNN
model = Net()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

Net(
  (conv1): Conv1d(1, 3, kernel_size=(102,), stride=(1,), padding=(1,))
  (conv2): Conv1d(3, 10, kernel_size=(24,), stride=(1,), padding=(1,))
  (conv3): Conv1d(10, 10, kernel_size=(11,), stride=(1,), padding=(1,))
  (conv4): Conv1d(10, 10, kernel_size=(9,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=250, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
)


the regularization, momentum, and learning rate parameters are set to 0.2, 3 × 10 −4 , and 0.7 respectively.

In [12]:
import torch.optim as optim

regularization = 0.2
momentum = 3e-4
lr = 0.7

criterion = nn.NLLLoss()

optmizer = optim.SGD(model.parameters(), lr= lr, momentum= momentum, weight_decay= regularization)

## Treino

In [22]:
def train_model(nEpochs = 10):
    train_loss_list =[]
    valid_loss_list = []

    valid_loss_min = np.Inf # Minimo valid loss

    for epoch in range(nEpochs):
        train_loss= 0
        validation_loss= 0

        model.train()
        for sample, target in train_loader:       
        # Zerar o gradiente
            optimizer.zero_grad()
        # Gera saída do modelo
            outputs = model(sample)
        # Calcula o erro
            loss = criterion(outputs, target)
            train_loss+=loss.item()*data.size(0)
        # Calcula os gradientes
            loss.backward()
        # Otimiza o modelo
            optimizer.step()
            
        model.eval()
        for data, target in valid_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)

        train_loss = train_loss/len(train_dataloader.dataset)
        train_loss_list.append(train_loss)
        valid_loss = valid_loss/len(train_dataloader.dataset)
        valid_loss_list.append(valid_loss)
      
        if (not epoch%5):
            print(f"Época: {epoch} \nLoss Treino: {train_loss}")
            print(f"Loss Teste: {valid_loss}")

        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min, valid_loss))
            torch.save(model.state_dict(), 'model_cifar.pt')
            valid_loss_min = valid_loss

    return train_loss_list, valid_loss_list


In [ ]:
trained_data, validated_data = train_model(60)

## Teste